In [2]:
# Import required Python libraries
import time
import base64
import hashlib
import hmac
import urllib.request
import json
import websocket
from websocket import create_connection
import requests
from dotenv import load_dotenv
import os
import _thread
import sys


load_dotenv()

True

# Fetch Websocket Authentication Token via Rest

In [3]:
# This code cell was taken directly from the kraken documentation shown here:
# https://support.kraken.com/hc/en-us/articles/360034437672-How-to-retrieve-a-WebSocket-authentication-token-Example-code-in-Python-3

# Copy/paste API public key and API private key as displayed in account management
api_key = os.getenv('API_KEY')
api_secret = os.getenv('PRIVATE_KEY')
 
# Variables (API method, nonce, and POST data)
api_path = '/0/private/GetWebSocketsToken'
api_nonce = str(int(time.time()*1000))
api_post = 'nonce=' + api_nonce
 
# Cryptographic hash algorithms
api_sha256 = hashlib.sha256(api_nonce.encode('utf-8') + api_post.encode('utf-8'))
api_hmac = hmac.new(base64.b64decode(api_secret), api_path.encode('utf-8') + api_sha256.digest(), hashlib.sha512)
 
# Encode signature into base64 format used in API-Sign value
api_signature = base64.b64encode(api_hmac.digest())
 
# HTTP request (POST)
api_request = urllib.request.Request('https://api.kraken.com/0/private/GetWebSocketsToken', api_post.encode('utf-8'))
api_request.add_header('API-Key', api_key)
api_request.add_header('API-Sign', api_signature)
api_response = urllib.request.urlopen(api_request).read().decode()

# Output API response
ws_token = json.loads(api_response)['result']['token']

# Create Websocket Connection

In [4]:
command = {
  "event": "subscribe",
  "pair": [
    "XBT/USD"
  ],
  "subscription": {
    "name": "trade"
  }
}

data = ""
    
# Define WebSocket callback functions
def ws_thread(*args):
    ws = websocket.WebSocketApp('wss://ws.kraken.com/', on_open=ws_open, on_message=ws_message)
    ws.run_forever()
    
    
def ws_open(ws):
    ws.send(json.dumps(command).encode('utf-8'))

def ws_message(ws, ws_data):
    data = json.loads(ws_data)
    print(data)

# Start new thread for WebSocket interface
_thread.start_new_thread(ws_thread, ())


# Just run for a minute for now 
# TODO: fix this
try:
    for i in range(60):
        time.sleep(1)
except KeyboardInterrupt:
    sys.exit(0)

{'connectionID': 18275597804541235006, 'event': 'systemStatus', 'status': 'online', 'version': '1.9.1'}
{'channelID': 337, 'channelName': 'trade', 'event': 'subscriptionStatus', 'pair': 'XBT/USD', 'status': 'subscribed', 'subscription': {'name': 'trade'}}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
[337, [['44342.80000', '0.00112775', '1704409841.640278', 's', 'l', '']], 'trade', 'XBT/USD']
{'event': 'heartbeat'}
{'event': 'heartbeat'}
[337, [['44342.80000', '0.00168000', '1704409843.494925', 's', 'm', '']], 'trade', 'XBT/USD']
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
{'event': 'heartbeat'}
[337, [['44342.80000', '0.00084192', '1704409854.685033', 's', 'l', '']], 'trade', 'XBT/USD']
[337, [['44342.80000', '0.00076614', '1704409855.300713', 's', 'l', '']], 'trade', 'XBT/USD']
{'event': 'heartbeat'}
